In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import plotly.express as px


from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

/Users/sannehoeken/Library/Python/3.9/lib/python/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# info on datafile and pre-trained model
input_path = 'HateWiC_T5Defs_MajorityLabels.csv'
sentence_column = 'T5generated_definition'
label_column = 'majority_binary_annotation'
id_column = 'id'

pretrained_model_name = 'sentence-transformers/all-mpnet-base-v2'
trained_model_dir = 'CL-model/final/'

In [3]:
def get_sentence_embeddings(data, sentence_column, model_path, embedding_name, layer_no=None):

    if layer_no == None:
        model = SentenceTransformer(model_path).cpu() # device='mps' gives error
        embeddings = model.encode(data[sentence_column], show_progress_bar=True)
    
    else:
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
        encoded_input = tokenizer(list(data[sentence_column]), padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)

        hidden_states = model_output[-1]
        layer_index = min(max(0, layer_no-1), model.config.num_hidden_layers-1)
        token_embeddings = hidden_states[layer_index] #.squeeze(0) ???
        # To test (should result in same output as if layer_no == None code)
        # token_embeddings = model_output[0]

        # mean pooling and normalization
        input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
        embeddings = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        embeddings = F.normalize(embeddings, p=2, dim=1)
        embeddings = [t.numpy() for t in list(embeddings)]


    data[embedding_name] = list(embeddings)
    
    return data


def get_tsne_data(data, embedding_column):
   
    tsne = TSNE(verbose=1, random_state=12)
    embeddings = np.array(list(data[embedding_column]))
    embedded_space = tsne.fit_transform(embeddings)

    data[embedding_column+'-tsne-comp-1'] = embedded_space[:,0]
    data[embedding_column+'-tsne-comp-2'] = embedded_space[:,1]

    return data


def plot(data, x_column, y_column, label_column, id_column, plot_title=""):
    
    plot_df = pd.DataFrame()
    plot_df["x"] = data[x_column]
    plot_df["y"] = data[y_column]
    plot_df["label"] = data[label_column].astype(str)
    plot_df["data_id"] = data[id_column]

    fig = px.scatter(plot_df, x="x", y="y", color="label", title=plot_title,
                     hover_data={'data_id': True, 'x':False, 'y': False})
    fig.update_layout(yaxis_title=None, xaxis_title=None)
    fig.show()

In [4]:
# load data
data = pd.read_csv(input_path, sep=';')

In [5]:
# get sentence embedding and T-SNE projection data for pretrained model
data = get_sentence_embeddings(data, sentence_column, pretrained_model_name, 'pretrained_embeddings')
data = get_tsne_data(data, 'pretrained_embeddings')

/Users/sannehoeken/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 121/121 [00:19<00:00,  6.30it/s]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3845 samples in 0.001s...
[t-SNE] Computed neighbors for 3845 samples in 0.464s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3845
[t-SNE] Computed conditional probabilities for sample 2000 / 3845
[t-SNE] Computed conditional probabilities for sample 3000 / 3845
[t-SNE] Computed conditional probabilities for sample 3845 / 3845
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 84.565857
[t-SNE] KL divergence after 1000 iterations: 1.427646


In [6]:
# plot the T-SNE projections of the pretrained model
x_column = 'pretrained_embeddings-tsne-comp-1'
y_column = 'pretrained_embeddings-tsne-comp-2'
plot_title = 'T-SNE projection of pretrained sentence embedding space'
plot(data, x_column, y_column, label_column, id_column, plot_title=plot_title)

In [7]:
# get sentence embedding and T-SNE projection data for contrastive learned model
data = get_sentence_embeddings(data, sentence_column, trained_model_dir, 'trained_embeddings')
data = get_tsne_data(data, 'trained_embeddings')

Batches: 100%|██████████| 121/121 [00:18<00:00,  6.45it/s]


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3845 samples in 0.001s...
[t-SNE] Computed neighbors for 3845 samples in 0.224s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3845
[t-SNE] Computed conditional probabilities for sample 2000 / 3845
[t-SNE] Computed conditional probabilities for sample 3000 / 3845
[t-SNE] Computed conditional probabilities for sample 3845 / 3845
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 61.591293
[t-SNE] KL divergence after 1000 iterations: 0.645946


In [8]:
# plot the T-SNE projections of the contrastive learned model
x_column = 'trained_embeddings-tsne-comp-1'
y_column = 'trained_embeddings-tsne-comp-2'
plot_title = 'T-SNE projection of contrastive learned sentence embedding space'
plot(data, x_column, y_column, label_column, id_column, plot_title=plot_title)

In [9]:
# get sentence embedding and T-SNE projection data for a specific hidden layer of the trained model, e.g the 12th
layer_no = 12
data = get_sentence_embeddings(data, sentence_column, trained_model_dir, f'trained_embeddings_layer{layer_no}', layer_no=layer_no)
data = get_tsne_data(data, f'trained_embeddings_layer{layer_no}')

# and plot
x_column = f'trained_embeddings_layer{layer_no}-tsne-comp-1'
y_column = f'trained_embeddings_layer{layer_no}-tsne-comp-2'
plot_title = f'T-SNE projection of contrastive learned sentence embedding space - Layer {layer_no}'
plot(data, x_column, y_column, label_column, id_column, plot_title=plot_title)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3845 samples in 0.019s...
[t-SNE] Computed neighbors for 3845 samples in 0.255s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3845
[t-SNE] Computed conditional probabilities for sample 2000 / 3845
[t-SNE] Computed conditional probabilities for sample 3000 / 3845
[t-SNE] Computed conditional probabilities for sample 3845 / 3845
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 67.666107
[t-SNE] KL divergence after 1000 iterations: 0.895267


In [10]:
# get sentence embedding and T-SNE projection data for a specific hidden layer of the trained model, e.g the first
layer_no = 1
data = get_sentence_embeddings(data, sentence_column, trained_model_dir, f'trained_embeddings_layer{layer_no}', layer_no=layer_no)
data = get_tsne_data(data, f'trained_embeddings_layer{layer_no}')

# and plot
x_column = f'trained_embeddings_layer{layer_no}-tsne-comp-1'
y_column = f'trained_embeddings_layer{layer_no}-tsne-comp-2'
plot_title = f'T-SNE projection of contrastive learned sentence embedding space - Layer {layer_no}'
plot(data, x_column, y_column, label_column, id_column, plot_title=plot_title)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 3845 samples in 0.007s...
[t-SNE] Computed neighbors for 3845 samples in 0.232s...
[t-SNE] Computed conditional probabilities for sample 1000 / 3845
[t-SNE] Computed conditional probabilities for sample 2000 / 3845
[t-SNE] Computed conditional probabilities for sample 3000 / 3845
[t-SNE] Computed conditional probabilities for sample 3845 / 3845
[t-SNE] Mean sigma: 0.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.784393
[t-SNE] KL divergence after 1000 iterations: 1.428479


In [11]:
# get the train-test-splits as the contrastive learning model was also trained with (so don't change the random_state)
train_data, dev_test_data = train_test_split(data, train_size=0.8, random_state=12)
dev_data, test_data = train_test_split(dev_test_data, train_size=0.5, random_state=12)

# plot only one of the splits, e.g. test
x_column = 'trained_embeddings-tsne-comp-1'
y_column = 'trained_embeddings-tsne-comp-2'
plot_title = 'T-SNE projection of contrastive learned sentence embedding space - Test data only'
plot(test_data, x_column, y_column, label_column, id_column, plot_title=plot_title)